In [93]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [94]:
path='./data/'

users = pd.read_csv(path+'users.csv')
books = pd.read_csv(path+'books.csv')
ratings = pd.read_csv(path+'train_ratings.csv')
test_ratings = pd.read_csv(path+'test_ratings.csv')

print('users shape: ', users.shape)
print('books shape: ', books.shape)
print('ratings shape: ', ratings.shape)
print('test_ratings shape:', test_ratings.shape)

users shape:  (68092, 3)
books shape:  (149570, 10)
ratings shape:  (306795, 3)
test_ratings shape: (76699, 3)


category 전처리 -> category_high 분류 명확히

In [95]:
import re
books.loc[books['category'].notna(), 'category'] = books[books['category'].notna()]['category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip()).str.lower()

In [97]:
from collections import defaultdict
import operator

cate = list(books['category'].value_counts().index)
cate_list = defaultdict(int)

for ca in cate:
    for c in ca.lower().split(' '):
        cate_list[c] += 1

sort_cate = dict(sorted(cate_list.items(), key=operator.itemgetter(1), reverse=True))
sort_cate

{'and': 197,
 'fictitious': 169,
 'character': 167,
 'fiction': 132,
 'american': 92,
 'children': 90,
 'stories': 79,
 'in': 74,
 'literature': 62,
 's': 60,
 'of': 59,
 'english': 57,
 'language': 44,
 'books': 40,
 'computer': 38,
 'art': 36,
 'women': 36,
 'psychology': 34,
 'authors': 34,
 'poetry': 33,
 'television': 33,
 'england': 32,
 'african': 29,
 'french': 29,
 'cookery': 27,
 'program': 27,
 'science': 26,
 'german': 24,
 'picture': 24,
 'cooking': 22,
 'the': 22,
 'life': 21,
 'motion': 21,
 'canadian': 21,
 'imaginary': 21,
 'business': 20,
 'animals': 20,
 'christian': 19,
 'child': 19,
 'indians': 19,
 'new': 19,
 'human': 18,
 'animal': 18,
 'drama': 17,
 'n': 17,
 'characters': 17,
 'book': 17,
 'industry': 17,
 'humor': 15,
 'mystery': 15,
 'americans': 15,
 'australian': 15,
 'belletristische': 15,
 'darstellung': 15,
 'social': 14,
 'education': 14,
 'detective': 14,
 'short': 14,
 'behavior': 14,
 'place': 14,
 'spanish': 14,
 'chinese': 14,
 'family': 13,
 '194

In [98]:
for category, nums in sort_cate.items():
    if nums >= 10:
        print("############# CATEGORY ############### : ", category)
        print(books[books['category'].str.contains(category,na=False)]['category'].value_counts())
        # print(books[books])
        print('----------')
    # print(category)
    # books['category'].str.contains(category,na=False)

############# CATEGORY ############### :  and
detective and mystery stories       257
brothers and sisters                 84
england                              65
adventure and adventurers            51
american wit and humor               45
                                   ... 
yorkshire england                     1
foster grandparents                   1
architecture and climate              1
bed and breakfast accommodations      1
england southern                      1
Name: category, Length: 325, dtype: int64
----------
############# CATEGORY ############### :  fictitious
baggins frodo fictitious character       12
dracula count fictitious character        9
boxcar children fictitious characters     8
covenant thomas fictitious character      8
dent arthur fictitious character          7
                                         ..
murry family fictitious characters        1
chase angela fictitious character         1
baby roo fictitious character             1
muppets fict

In [99]:
# 일일히 찍어서 수작업으로 상위 32개 단어 추출

common_cate = ['ficti', 'american english', 'child', 'stories', 'literature', 'language',
'computer', 'art', 'women', 'psychology', 'poetry', 'television drama', 'african', 
'french', 'cook', 'science', 'life', 'picture', 'business', 'animal', 'christian',
'character', 'humor wit', 'mystery', 'social', 'educat', 'family', 'self', 'philosophy', 
'photo', 'law', 'game']

len(common_cate)

32

In [100]:
for ccate in common_cate:
    for c in ccate.split():
        books.loc[books['category'].str.contains(c, na=False), 'category_high'] = ccate

In [101]:
books['category_high'].value_counts()

ficti               40754
art                  1598
humor wit            1309
social               1278
science              1200
business             1147
cook                 1125
family                988
computer              820
poetry                777
psychology            698
stories               671
self                  669
television drama      568
american english      493
philosophy            492
language              379
educat                366
mystery               296
character             295
game                  294
child                 260
animal                225
life                  183
photo                 178
literature            160
christian             142
law                   130
african               108
french                104
picture                86
women                  50
Name: category_high, dtype: int64

In [102]:
import matplotlib.pyplot as plt
from PIL import Image
import requests

image_path = './data/'

for i in range(len(books)):
    file_path = image_path + books['img_path'][i]
    img = np.asarray(Image.open(file_path))
    if len(img) == 1:
        books.loc[books.index[i], 'img_path'] = ''

In [103]:
target_cate_img = books[(books['category_high'].notna()) & (books['img_path'] != '')]

In [104]:
train_data = list()
val_data = list()
train_label = list()
val_label = list()

common_cate_dict = {label: i for i, label in enumerate(common_cate)}

In [105]:
for cate in common_cate:
    imgs = []
    img_per_class = books[(books['category_high'] == cate) & (books['img_path'] != '')]['img_path']
    num_img_per_class = len(img_per_class)
    imgs.extend(('./data/' + img_per_class.values))
    
    # fiction undersampling 위함
    if cate == 'ficti':
       num_img_per_class = 2000
       
    for i, img in enumerate(imgs):
      if i > 2000: break
         
      if i < int(0.8 * num_img_per_class):
        train_data.append(Image.open(img).convert('RGB'))
        train_label.append(common_cate_dict[cate])
      else:
        val_data.append(Image.open(img).convert('RGB'))
        val_label.append(common_cate_dict[cate])

In [106]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import itertools
import matplotlib.pyplot as plt
import os
import numpy as np
import torch.nn as nn
import pandas as pd
import random

from PIL import Image

transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])])

In [107]:
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def num_classes(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]
        if self.transform:
          img = self.transform(img)
        return img, label

In [108]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True, norm="bnorm", relu=True):
        super().__init__()

        layers = []
        layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                             kernel_size=kernel_size, stride=stride, padding=padding,
                             bias=bias)]

        if norm == "bnorm":
            layers += [nn.BatchNorm2d(num_features=out_channels)]

        if relu:
            layers += [nn.ReLU()]

        self.cbr = nn.Sequential(*layers)

    def forward(self, x):
        return self.cbr(x)

In [109]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, 
                 padding=1, bias=True, norm="bnorm", short_cut=False, relu=True, init_block=False):
        super().__init__()

        layers = []


        if init_block:
          init_stride = 2
        else:
          init_stride = stride

        # 1st conv
        layers += [ConvBlock(in_channels=in_channels, out_channels=out_channels,
                         kernel_size=kernel_size, stride=init_stride, padding=padding,
                         bias=bias, norm=norm, relu=relu)]

        # 2nd conv
        layers += [ConvBlock(in_channels=out_channels, out_channels=out_channels,
                         kernel_size=kernel_size, stride=stride, padding=padding,
                         bias=bias, norm=norm, relu=False)]

        self.resblk = nn.Sequential(*layers)
        
        
        self.short_cut = nn.Conv2d(in_channels, out_channels, (1,1), stride=2)

    def forward(self, x, short_cut=False):
        if short_cut:
            return self.short_cut(x) + self.resblk(x)
        else:
            return x + self.resblk(x) # residual connection

In [110]:
class ResNet(nn.Module):
    def __init__(self, in_channels, out_channels, nker=64, norm="bnorm", nblk=[3,4,6,3]):
        super(ResNet, self).__init__()

        self.enc = ConvBlock(in_channels, nker, kernel_size=7, stride=2, padding=1, bias=True, norm=None, relu=True)
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        res_1 = ResBlock(nker, nker, kernel_size=3, stride=1, padding=1, bias=True, norm=norm, relu=True)
        self.res_1 = nn.Sequential(*[res_1 for _ in range(nblk[0])])

        res_2 = ResBlock(nker*2, nker*2, kernel_size=3, stride=1, padding=1, bias=True, norm=norm, relu=True)
        self.res_2_up = ResBlock(nker, nker*2, kernel_size=3, stride=1, padding=1, bias=True, norm=norm, relu=True, init_block=True)
        self.res_2 = nn.Sequential(*[res_2 for _ in range(nblk[1]-1)])

        res_3 = ResBlock(nker*2*2, nker*2*2, kernel_size=3, stride=1, padding=1, bias=True, norm=norm, relu=True)
        self.res_3_up = ResBlock(nker*2, nker*2*2, kernel_size=3, stride=1, padding=1, bias=True, norm=norm, relu=True, init_block=True)
        self.res_3 = nn.Sequential(*[res_3 for _ in range(nblk[2]-1)])

        res_4 = ResBlock(nker*2*2*2, nker*2*2*2, kernel_size=3, stride=1, padding=1, bias=True, norm=norm, relu=True, init_block=True)
        self.res_4_up = ResBlock(nker*2*2, nker*2*2*2, kernel_size=3, stride=1, padding=1, bias=True, norm=norm, relu=True)
        self.res_4 = nn.Sequential(*[res_4 for _ in range(nblk[3]-1)])

        self.avg_pooling = nn.AdaptiveAvgPool2d(output_size=1)
        self.fc = nn.Linear(nker*2*2*2, 32)

    def forward(self, x):
        x = self.enc(x)
        x = self.max_pool(x)
        x = self.res_1(x)
        x = self.max_pool(x)

        x = self.res_2_up(x, short_cut=True)
        x = self.res_2(x)
        x = self.max_pool(x)

        x = self.res_3_up(x, short_cut=True)
        x = self.res_3(x)
        x = self.max_pool(x)

        x = self.res_4_up(x, short_cut=True)
        x = self.res_4(x)
        
        x = self.avg_pooling(x)
        x = x.view(x.shape[0], -1)
        out = self.fc(x)
        return out

In [111]:
from torch.optim import Adam

# Build user-defined ResNet model
model_scratch = ResNet(3, 32)

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model_scratch.parameters(), lr=1e-4)

In [112]:
log_dir ='./log'

In [113]:
# quickdraw train/validatoin dataset and dataloader
b_train_dataset = CustomDataset(train_data, train_label, transform)
b_val_dataset = CustomDataset(val_data, val_label, transform)

b_train_dataloader = DataLoader(b_train_dataset, batch_size=4, shuffle=True, drop_last=True)
b_val_dataloader = DataLoader(b_val_dataset, batch_size=4, shuffle=True, drop_last=True)

In [114]:
# Misc
class AverageMeter(object):
  """Computes and stores the average and current value"""
  def __init__(self):
      self.reset()

  def reset(self):
    self.val = 0
    self.avg = 0
    self.sum = 0
    self.count = 0

  def update(self, val, n=1):
    self.val = val
    self.sum += val * n
    self.count += n
    self.avg = self.sum / self.count

In [115]:
model_scratch = model_scratch.cuda()

In [116]:
# Main
os.makedirs(log_dir, exist_ok=True)

with open(os.path.join(log_dir, 'scratch_train_log.csv'), 'w') as log:
  # Training
  model_scratch.train()
  for iter, (img, label) in enumerate(b_train_dataloader):
    # optimizer에 저장된 미분값을 0으로 초기화
    optimizer.zero_grad()

    img = img.cuda()
    label = label.cuda()

    # 모델에 이미지 forward
    pred_logit = model_scratch(img)

    # loss 값 계산
    loss = criterion(pred_logit, label)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Accuracy 계산
    pred_label = torch.argmax(pred_logit, 1)
    acc = (pred_label == label).sum().item() / len(img)

    train_loss = loss.item()
    train_acc = acc

    # Validation 
    if (iter % 20 == 0) or (iter == len(b_train_dataloader)-1):
      model_scratch.eval()
      valid_loss, valid_acc = AverageMeter(), AverageMeter()

      for img, label in b_val_dataloader:
        # Validation에 사용하기 위한 image, label 처리 (필요한 경우, data type도 변경해주세요)
        img, label = img.float().cuda(), label.long().cuda()

        # 모델에 이미지 forward (gradient 계산 X)
        with torch.no_grad():
          pred_logit = model_scratch(img)

        # loss 값 계산
        loss = criterion(pred_logit, label)

        # Accuracy 계산
        pred_label = torch.argmax(pred_logit, 1)
        acc = (pred_label == label).sum().item() / len(img)

        valid_loss.update(loss.item(), len(img))
        valid_acc.update(acc, len(img))

      valid_loss = valid_loss.avg
      valid_acc = valid_acc.avg

      print("Iter [%3d/%3d] | Train Loss %.4f | Train Acc %.4f | Valid Loss %.4f | Valid Acc %.4f" %
            (iter, len(b_train_dataloader), train_loss, train_acc, valid_loss, valid_acc))
      
      # Train Log Writing
      log.write('%d,%.4f,%.4f,%.4f,%.4f\n'%(iter, train_loss, train_acc, valid_loss, valid_acc))

Iter [  0/3022] | Train Loss 5.3107 | Train Acc 0.0000 | Valid Loss 3.4666 | Valid Acc 0.0290
Iter [ 20/3022] | Train Loss 3.9881 | Train Acc 0.0000 | Valid Loss 3.2162 | Valid Acc 0.1337
Iter [ 40/3022] | Train Loss 2.6540 | Train Acc 0.2500 | Valid Loss 3.1464 | Valid Acc 0.1314
Iter [ 60/3022] | Train Loss 3.1739 | Train Acc 0.0000 | Valid Loss 3.1223 | Valid Acc 0.1314
Iter [ 80/3022] | Train Loss 2.6286 | Train Acc 0.5000 | Valid Loss 3.1785 | Valid Acc 0.1186
Iter [100/3022] | Train Loss 3.0888 | Train Acc 0.0000 | Valid Loss 3.1838 | Valid Acc 0.0623
Iter [120/3022] | Train Loss 3.1980 | Train Acc 0.0000 | Valid Loss 3.1130 | Valid Acc 0.0870
Iter [140/3022] | Train Loss 2.3244 | Train Acc 0.5000 | Valid Loss 3.1333 | Valid Acc 0.1324
Iter [160/3022] | Train Loss 3.1765 | Train Acc 0.2500 | Valid Loss 3.1086 | Valid Acc 0.1301
Iter [180/3022] | Train Loss 3.1170 | Train Acc 0.0000 | Valid Loss 3.1509 | Valid Acc 0.1044
Iter [200/3022] | Train Loss 2.8645 | Train Acc 0.2500 | Val

In [117]:
torch.save(model_scratch.state_dict(), "./model_scratch_v4.pt")

In [118]:
imgs_for_pred_list = []
imgs_for_pred = books[(books['img_path'] != '') & ((books['category_high'].isna()))]['img_path']
imgs_for_pred_list.extend(('./data/' + imgs_for_pred.values))

len(imgs_for_pred_list)

imgs_for_prediction = []
for img in imgs_for_pred_list:
    imgs_for_prediction.append(Image.open(img).convert('RGB'))

# quickdraw train/validatoin dataset and dataloader
b_pred = CustomDataset(imgs_for_prediction, [-1]*len(imgs_for_prediction), transform)
b_pred_dataloader = DataLoader(b_pred, batch_size=4, shuffle=False, drop_last=True)

In [119]:
convert_class_dict = {v:k for k,v in common_cate_dict.items()}

for i, (img, label) in enumerate(b_pred_dataloader):
    img, label = img.float().cuda(), label.long().cuda()
    
    
    with torch.no_grad():
        pred_logit = model_scratch(img)
    
    pred_label = torch.argmax(pred_logit, 1)

    for idx, l in enumerate(pred_label):
        books.loc[(books['img_path'] == imgs_for_pred.values[4 * i + idx]), 'category_high'] = convert_class_dict.get(l.item())

publisher 분류

In [120]:
publisher_dict=(books['publisher'].value_counts()).to_dict()
publisher_count_df= pd.DataFrame(list(publisher_dict.items()),columns = ['publisher','count'])

publisher_count_df = publisher_count_df.sort_values(by=['count'], ascending = False)

modify_list = publisher_count_df[publisher_count_df['count']>1].publisher.values

for publisher in modify_list:
    try:
        number = books[books['publisher']==publisher]['isbn'].apply(lambda x: x[:4]).value_counts().index[0]
        right_publisher = books[books['isbn'].apply(lambda x: x[:4])==number]['publisher'].value_counts().index[0]
        books.loc[books[books['isbn'].apply(lambda x: x[:4])==number].index,'publisher'] = right_publisher
    except: 
        pass


In [121]:
books.isna().sum()

isbn                       0
book_title                 0
book_author                0
year_of_publication        0
publisher                  0
img_url                    0
language               67227
category               68851
summary                67227
img_path                   0
category_high          31415
dtype: int64

2차 : author <br>
3차 : publisher

In [123]:
BA = list(books[books['category_high'].notna()]['book_author'].value_counts().index)

for author in BA:
    if not (books[books['book_author'] == author]['category_high'].value_counts().empty):
        books.loc[(books['img_path'] == '') & (books['category_high'].isna()) & (books['book_author'] == author), 'category_high'] = books[books['book_author'] == author]['category_high'].value_counts().index[0]

pub = books['publisher'].unique()

for p in pub:
    if not (books[(books['publisher'] == p) & (books['category_high'].notna())]['category_high'].value_counts().empty):
        books.loc[(books['publisher'] == p) & (books['category_high'].isna()), 'category_high'] = books[(books['publisher'] == p) & (books['category_high'].notna())]['category_high'].value_counts().index[0]

4차 : others

In [124]:
books.loc[books['category_high'].isna(), 'category_high'] = 'others'

In [125]:
books['category_high'].value_counts()

ficti               110539
humor wit             7899
family                7052
computer              4619
cook                  2416
psychology            2213
art                   1892
business              1580
social                1398
science               1350
poetry                1252
stories                785
self                   728
television drama       720
philosophy             570
american english       547
language               404
educat                 392
mystery                358
character              331
game                   330
others                 321
child                  308
animal                 264
photo                  198
life                   197
literature             175
christian              174
law                    140
picture                132
african                117
french                 110
women                   59
Name: category_high, dtype: int64

In [126]:
books.to_csv('./data/v8/books.csv', index=False)